# 01. 데이터 수집 (Data Collection)

**목표**: KOBIS API와 Naver 검색 트렌드 데이터 수집

## 📋 수집 계획

1. **KOBIS API**
   - 최근 2-3년간 개봉 영화 정보
   - 목표: 100개 이상 영화
   - 수집 항목: 영화명, 개봉일, 장르, 감독, 배우, 배급사, 등급, 총 관객수

2. **Naver 검색 트렌드**
   - 각 영화별 검색량 추이
   - 기간: 개봉 4주 전 ~ 1주 후
   - 주간/일간 검색량 데이터

## 1. 라이브러리 및 설정

In [15]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 프로젝트 루트 경로 추가
sys.path.append('..')

from utils.data_collection import KOBISCollector, NaverTrendCollector, save_data, load_data

print("라이브러리 로드 완료!")

라이브러리 로드 완료!


## 2. KOBIS API 데이터 수집

### 2.1 API 키 설정
- 발급 사이트: https://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

In [24]:
# KOBIS API 키 입력
KOBIS_API_KEY = "4beefdb0727a4fae075456bfdd7c0a2b"  # 여기에 발급받은 API 키 입력

# Collector 초기화
kobis_collector = KOBISCollector(api_key=KOBIS_API_KEY)

### 2.2 박스오피스 데이터 수집

In [17]:
# 데이터 수집 기간 설정 (최근 2년)
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 12, 31)

# 박스오피스 데이터 수집
print("박스오피스 데이터 수집 시작...")
boxoffice_data = kobis_collector.collect_boxoffice_data(start_date, end_date, delay=0.1)

# DataFrame 변환
boxoffice_df = pd.DataFrame(boxoffice_data)
print(f"\n수집 완료! 총 {len(boxoffice_df)} 건의 데이터")
boxoffice_df.head()

박스오피스 데이터 수집 시작...


100%|██████████| 1827/1827 [19:21<00:00,  1.57it/s]


수집 완료! 총 16640 건의 데이터


,date,rank,movieCd,movieNm,openDt,salesAmt,audiCnt,audiAcc,scrnCnt,showCnt
0,2020-01-01,1,20192206,백두산,2019-12-19,4430968920,498802,6789304,1236,5845
1,2020-01-01,2,20184571,천문: 하늘에 묻는다,2019-12-26,2092505020,239149,1204773,929,3439
2,2020-01-01,3,20198414,시동,2019-12-18,1691272350,190455,2716441,801,2997
3,2020-01-01,4,20196272,미드웨이,2019-12-31,1561075860,174407,307780,810,2614
4,2020-01-01,5,20197803,겨울왕국 2,2019-11-21,727968600,83529,13452593,665,1459


In [18]:
# 데이터 정보 확인
print("=== 박스오피스 데이터 정보 ===")
print(f"행 개수: {len(boxoffice_df)}")
print(f"컬럼: {list(boxoffice_df.columns)}")
print(f"\n고유 영화 개수: {boxoffice_df['movieNm'].nunique()}")
boxoffice_df.info()

=== 박스오피스 데이터 정보 ===
행 개수: 16640
컬럼: ['date', 'rank', 'movieCd', 'movieNm', 'openDt', 'salesAmt', 'audiCnt', 'audiAcc', 'scrnCnt', 'showCnt']

고유 영화 개수: 1252
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16640 entries, 0 to 16639
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      16640 non-null  object
 1   rank      16640 non-null  object
 2   movieCd   16640 non-null  object
 3   movieNm   16640 non-null  object
 4   openDt    16640 non-null  object
 5   salesAmt  16640 non-null  int64 
 6   audiCnt   16640 non-null  int64 
 7   audiAcc   16640 non-null  int64 
 8   scrnCnt   16640 non-null  int64 
 9   showCnt   16640 non-null  int64 
dtypes: int64(5), object(5)
memory usage: 1.3+ MB


### 2.3 영화 상세 정보 수집

In [25]:
# 고유 영화 코드 추출
unique_movie_codes = boxoffice_df['movieCd'].unique().tolist()
print(f"수집할 영화 개수: {len(unique_movie_codes)}")

# 영화 상세 정보 수집
print("\n영화 상세 정보 수집 시작...")
movie_details = kobis_collector.collect_movie_details(unique_movie_codes, delay=0.1)

# DataFrame 변환
movie_details_df = pd.DataFrame(movie_details)
print(f"\n수집 완료! 총 {len(movie_details_df)} 편의 영화 상세 정보")
movie_details_df.head()

수집할 영화 개수: 1256

영화 상세 정보 수집 시작...


100%|██████████| 1256/1256 [09:32<00:00,  2.20it/s]


수집 완료! 총 1256 편의 영화 상세 정보


,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,nations,genres,directors,actors,watchGradeNm,distributors
0,20192206,백두산,ASHFALL,2019,20191219,장편,한국,"어드벤처,드라마","김병서,이해준","이병헌,하정우,마동석,전혜진,배수지",12세이상관람가,"(주)씨제이이엔엠,(주)덱스터스튜디오"
1,20184571,천문: 하늘에 묻는다,Forbidden Dream,2019,20191226,장편,한국,사극,허진호,"최민식,한석규,신구,김홍파,허준호",12세이상관람가,롯데컬처웍스(주)롯데엔터테인먼트
2,20198414,시동,START-UP,2019,20191218,장편,한국,드라마,최정열,"마동석,박정민,정해인,염정아,최성은",15세이상관람가,(주)넥스트엔터테인먼트월드(NEW)
3,20196272,미드웨이,Midway,2019,20191231,장편,미국,"액션,드라마",롤랜드 에머리히,"에드 스크레인,패트릭 윌슨,우디 해럴슨,루크 에반스,아사노 타다노부",15세이상관람가,(주)누리픽쳐스
4,20197803,겨울왕국 2,Frozen 2,2019,20191121,장편,미국,애니메이션,"제니퍼 리,크리스 벅","크리스틴 벨,이디나 멘젤,조시 게드,조나단 그로프",전체관람가,월트디즈니컴퍼니코리아 유한책임회사


In [26]:
# 상세 정보 확인
print("=== 영화 상세 정보 ===")
movie_details_df.info()
print("\n샘플 데이터:")
movie_details_df.head(10)

=== 영화 상세 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1256 entries, 0 to 1255
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieCd       1256 non-null   object
 1   movieNm       1256 non-null   object
 2   movieNmEn     1256 non-null   object
 3   prdtYear      1256 non-null   object
 4   openDt        1256 non-null   object
 5   typeNm        1256 non-null   object
 6   nations       1256 non-null   object
 7   genres        1256 non-null   object
 8   directors     1256 non-null   object
 9   actors        1256 non-null   object
 10  watchGradeNm  1242 non-null   object
 11  distributors  1256 non-null   object
dtypes: object(12)
memory usage: 117.9+ KB

샘플 데이터:


,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,nations,genres,directors,actors,watchGradeNm,distributors
0,20192206,백두산,ASHFALL,2019,20191219,장편,한국,"어드벤처,드라마","김병서,이해준","이병헌,하정우,마동석,전혜진,배수지",12세이상관람가,"(주)씨제이이엔엠,(주)덱스터스튜디오"
1,20184571,천문: 하늘에 묻는다,Forbidden Dream,2019,20191226,장편,한국,사극,허진호,"최민식,한석규,신구,김홍파,허준호",12세이상관람가,롯데컬처웍스(주)롯데엔터테인먼트
2,20198414,시동,START-UP,2019,20191218,장편,한국,드라마,최정열,"마동석,박정민,정해인,염정아,최성은",15세이상관람가,(주)넥스트엔터테인먼트월드(NEW)
3,20196272,미드웨이,Midway,2019,20191231,장편,미국,"액션,드라마",롤랜드 에머리히,"에드 스크레인,패트릭 윌슨,우디 해럴슨,루크 에반스,아사노 타다노부",15세이상관람가,(주)누리픽쳐스
4,20197803,겨울왕국 2,Frozen 2,2019,20191121,장편,미국,애니메이션,"제니퍼 리,크리스 벅","크리스틴 벨,이디나 멘젤,조시 게드,조나단 그로프",전체관람가,월트디즈니컴퍼니코리아 유한책임회사
5,20198374,신비아파트 극장판 하늘도깨비 대 요르문간드,The Haunted House: The Sky Ghost VS Jormungandr,2019,20191219,장편,한국,애니메이션,변영규,"김영은,김채하,신용우,양정화,김현지",전체관람가,(주)씨제이이엔엠
6,20192401,포드 V 페라리,FORD v FERRARI,2019,20191204,장편,미국,"액션,드라마",제임스 맨골드,"맷 데이먼,크리스찬 베일",12세이상관람가,월트디즈니컴퍼니코리아 유한책임회사
7,20191408,21 브릿지: 테러 셧다운,21 Bridges,2019,20200101,장편,미국,"스릴러,드라마,범죄",브라이언 커크,"채드윅 보스만,J.K. 시몬스,시에나 밀러,테일러 키취",15세이상관람가,"(주)드림팩트엔터테인먼트,(주)제이앤씨미디어그룹,TCO(주)더콘텐츠온"
8,20198430,나이브스 아웃,KNIVES OUT,2018,20191204,장편,미국,"미스터리,스릴러",라이언 존슨,"다니엘 크레이그,크리스 에반스,아나 디 아르마스,제이미 리 커티스,마이클 섀넌",12세이상관람가,(주)올스타엔터테인먼트
9,20199981,눈의 여왕4,The Snow Queen: Mirrorlands,2019,20191224,장편,러시아,"애니메이션,가족,판타지,어드벤처","로버트 렌스,알렉세이 트시칠린","양정화,박지윤,임채헌,박영재,민승우",전체관람가,(주)넥스트엔터테인먼트월드(NEW)


### 2.4 KOBIS 데이터 저장

In [27]:
# 데이터 저장
save_data(boxoffice_df, 'data/raw/kobis_boxoffice.csv')
save_data(movie_details_df, 'data/raw/kobis_movie_details.csv')

print("KOBIS 데이터 저장 완료!")

Data saved to data/raw/kobis_boxoffice.csv
Data saved to data/raw/kobis_movie_details.csv
KOBIS 데이터 저장 완료!


## 4. 데이터 수집 요약

In [30]:
print("\n" + "="*50)
print("데이터 수집 요약")
print("="*50)

print("\n1. KOBIS 박스오피스 데이터")
print(f"   - 행 개수: {len(boxoffice_df):,}")
print(f"   - 고유 영화: {boxoffice_df['movieNm'].nunique()}편")
print(f"   - 파일: data/raw/kobis_boxoffice.csv")

print("\n2. KOBIS 영화 상세 정보")
print(f"   - 행 개수: {len(movie_details_df):,}")
print(f"   - 파일: data/raw/kobis_movie_details.csv")

print("\n3. Naver 검색 트렌드")
print(f"   - 파일: data/raw/naver_search_trends.csv")

print("\n" + "="*50)
print("✅ 데이터 수집 완료!")
print("다음 단계: 02_eda.ipynb (탐색적 데이터 분석)")
print("="*50)


데이터 수집 요약

1. KOBIS 박스오피스 데이터
   - 행 개수: 16,640
   - 고유 영화: 1252편
   - 파일: data/raw/kobis_boxoffice.csv

2. KOBIS 영화 상세 정보
   - 행 개수: 1,256
   - 파일: data/raw/kobis_movie_details.csv

3. Naver 검색 트렌드
   - 파일: data/raw/naver_search_trends.csv

✅ 데이터 수집 완료!
다음 단계: 02_eda.ipynb (탐색적 데이터 분석)
